In [29]:
from pathlib import Path

import maples_dr
import pandas as pd
from ipywidgets import HTML, Dropdown, GridBox, Layout
from jppype import imshow, vscode_theme, sync_views
from maples_dr.dataset import BiomarkerField as Bio
from variability_study_utils import count_lesions, load_annotations_from_scratch, load_reannotations

vscode_theme()

HTML(value="<style>\n        .cell-output-ipywidget-background {\n                background: transparent !imp…

# Load Images

In [2]:
maples_dr.configure(
    maples_dr_path="../PATH/TO/MAPLES-DR/MAPLES-DR_v2.zip", messidor_path="../PATH/TO/MESSIDOR", image_format="bgr"
)

In [10]:
train_names = maples_dr.quick_api.GLOBAL_LOADER.image_names("train")
test_names = maples_dr.quick_api.GLOBAL_LOADER.image_names("test")
duplicates_names = maples_dr.quick_api.GLOBAL_LOADER.image_names("duplicates")

In [3]:
fromscratch_datasets = load_annotations_from_scratch()
images = fromscratch_datasets[0].keys()
published_dataset = maples_dr.quick_api.GLOBAL_LOADER.load_dataset(images)
refined_preannotations = load_reannotations()[0][images]

## Compute lesions count for all images

In [21]:
n_lesions = pd.DataFrame(index=images)

n_lesions["set"] = ["test" if n in test_names else "train" if n in train_names else "duplicates" for n in images]

n_lesions["DR"] = published_dataset.data["dr"]
n_lesions["ME"] = published_dataset.data["me"]

for l in [Bio.RED_LESIONS, Bio.BRIGHT_LESIONS]:
    n_lesions[l.value] = count_lesions(published_dataset, l, preannotation=False)
    n_lesions[l.value + "_pre"] = count_lesions(published_dataset, l, preannotation=True)
    n_lesions[l.value + "_pre_refined"] = count_lesions(refined_preannotations, l, preannotation=True)
    for i, r in enumerate("ABC"):
        n_lesions[l.value + "_" + r] = count_lesions(fromscratch_datasets[i], l, preannotation=False)

In [24]:
n_lesions.loc[n_lesions["set"] == "test"]

,set,DR,ME,redLesions,redLesions_pre,redLesions_pre_refined,redLesions_A,redLesions_B,redLesions_C,brightLesions,brightLesions_pre,brightLesions_pre_refined,brightLesions_A,brightLesions_B,brightLesions_C
20060522_46266_0100_PP,test,R1,M2,28,42,8,9,6,8,1,16,1,0,0,0
20060412_52668_0200_PP,test,R1,M0,45,33,45,50,5,20,10,4,8,2,2,7
20060523_49010_0100_PP,test,R1,M1,38,38,51,96,48,39,11,4,5,12,6,15
20060411_58221_0200_PP,test,R1,M2,48,58,52,47,35,33,23,21,8,27,0,18
20051020_62461_0100_PP,test,R0,M0,4,6,2,17,2,2,0,2,1,33,1,0
20051202_51677_0400_PP,test,R2,M2,59,63,41,168,17,67,62,93,56,113,20,53
20051116_44816_0400_PP,test,R1,M0,21,16,6,5,3,6,4,2,2,14,10,4
20060412_59400_0200_PP,test,R1,M1,8,30,22,15,6,8,14,108,12,11,2,7
20060412_61000_0200_PP,test,R1,M2,53,39,39,28,12,18,2,18,4,3,0,3
20060412_60895_0200_PP,test,R1,M2,7,18,19,12,11,7,11,12,8,7,15,7


In [26]:
images = n_lesions.loc[n_lesions["set"] == "test"].index

## Review refined preannotations for the selected images

In [55]:
current_img = [images[0]]

In [52]:
img_selector = Dropdown(
    options=images,
    description="Biomarker:",
    layout=Layout(width="auto"),
    value=current_img[0],
)

s = published_dataset[images[0]]

view1 = imshow(s["fundus"])
view2 = imshow(s["fundus"])
view3 = imshow(s["fundus"])
view4 = imshow(s["fundus"])


def set_img(img):
    if isinstance(img, dict):
        img = img["new"]
    current_img[0] = img
    s_published = published_dataset[img]
    s_refined = refined_preannotations[img]

    for v, l in zip(
        [view1, view2, view3, view4],
        [Bio.MICROANEURYSMS, Bio.HEMORRHAGES, Bio.EXUDATES, Bio.COTTON_WOOL_SPOTS],
        strict=True,
    ):
        diff = (
            s_published[l]
            + 2 * s_published.read_biomarker(l, pre_annotation=True)
            + 4 * s_refined.read_biomarker(l, pre_annotation=True)
        )
        v["background"] = s_published["fundus"]
        v.add_label(
            diff,
            name="diff",
            colormap={
                # --- In published but absent from refined ---
                1: "#75395e",  # Published, added by clinician
                2: "#8a8442",  # Removed from Published by clinician
                3: "#44428a",  # Published, kept from pre-annotation
                # --- In refined but not in published ---
                4: "#ff5100",
                # --- In published and in refined ---
                5: "#00f710",  # Published, added by clinician
                6: "#ff0000",  # Removed from Published by clinician, retintroded in refined
                7: "#ffffff",  # Published, kept from pre-annotation
            },
        )


img_selector.observe(set_img, "value")
set_img(img_selector.value)

sync_views(view1, view2, view3, view4)

legend = HTML(
    """
<p style="color: var(--jppype-foreground-color); font-size: 14px; text-align: center;">
    Not in refined preannotations: <span style="display: inline-block"><span style="background-color: #75395e; width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Added by ret. </span>
    <span style="display: inline-block"><span style="background-color: #44428a; width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Kept from preannotation </span>
    <span style="display: inline-block"><span style="background-color: #8a8442; width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Removed by ret. </span>
</p>
<p  style="color: var(--jppype-foreground-color); font-size: 14px; text-align: center;">
    In refined preannotations:
    <span style="display: inline-block"><span style="background-color: #ff5100; width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Not added by ret. or pre. </span>

    <span style="display: inline-block"><span style="background-color: #ff0000; width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Removed by ret. </span>

    <span style="display: inline-block"><span style="background-color: #00f710; width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px"> </span> Added by ret. </span>
    <span style="display: inline-block"><span style="background-color: #ffffff; width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px"> </span> all agree </span>
</p>
"""
)

GridBox(
    [img_selector, legend, view1, view2, view3, view4],
    layout=Layout(grid_template_columns="repeat(2, 1fr)", grid_template_rows="auto 600px 600px"),
)

GridBox(children=(Dropdown(description='Biomarker:', index=7, layout=Layout(width='auto'), options=('20060522_…

In [54]:
20060412_52978_0200_PP

SyntaxError: invalid decimal literal (3507317493.py, line 1)